In [1]:
# pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# for visualizing the results
import numpy as np
import matplotlib.pyplot as plt

# for reading input data
import pandas as pd

# for parsing the FEN of chess positions
import re

In [2]:
def fen_to_bit_vector(fen):
    # piece placement - lowercase for black pieces, uppercase for white pieces. numbers represent consequtive spaces. / represents a new row 
    # active color - whose turn it is, either 'w' or 'b'
    # castling rights - which castling moves are still legal K or k for kingside and Q or q for queenside, '-' if no legal castling moves for either player
    # en passant - if the last move was a pawn moving up two squares, this is the space behind the square for the purposes of en passant
    # halfmove clock - number of moves without a pawn move or piece capture, after 50 of which the game is a draw
    # fullmove number - number of full turns starting at 1, increments after black's move

    # Example FEN of starting position
    # rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
    
    parts = re.split(" ", fen)
    piece_placement = re.split("/", parts[0])
    active_color = parts[1]
    castling_rights = parts[2]
    en_passant = parts[3]
    halfmove_clock = int(parts[4])
    fullmove_clock = int(parts[5])

    bit_vector = np.zeros((13, 8, 8), dtype=np.uint8)
    
    # piece to layer structure taken from reference [1]
    piece_to_layer = {
        'R': 1,
        'N': 2,
        'B': 3,
        'Q': 4,
        'K': 5,
        'P': 6,
        'p': 7,
        'k': 8,
        'q': 9,
        'b': 10,
        'n': 11,
        'r': 12
    }
    
    castling = {
        'K': (7,7),
        'Q': (7,0),
        'k': (0,7),
        'q': (0,0),
    }

    for r, row in enumerate(piece_placement):
        c = 0
        for piece in row:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece], r, c] = 1
                c += 1
            else:
                c += int(piece)
    
    if en_passant != '-':
        bit_vector[0, ord(en_passant[0]) - ord('a'), int(en_passant[1]) - 1] = 1
    
    if castling_rights != '-':
        for char in castling_rights:
            bit_vector[0, castling[char][0], castling[char][1]] = 1
    
    if active_color == 'w':
        bit_vector[0, 7, 4] = 1
    else:
        bit_vector[0, 0, 4] = 1

    if halfmove_clock > 0:
        c = 7
        while halfmove_clock > 0:
            bit_vector[0, 3, c] = halfmove_clock%2
            halfmove_clock = halfmove_clock // 2
            c -= 1
            if c < 0:
                break

    if fullmove_clock > 0:
        c = 7
        while fullmove_clock > 0:
            bit_vector[0, 4, c] = fullmove_clock%2
            fullmove_clock = fullmove_clock // 2
            c -= 1
            if c < 0:
                break

    return bit_vector

In [9]:
def fen_to_board_vector(fen):
    """
    We need to have a smaller representation of the board for the network to train on.
    """
    # piece placement - lowercase for black pieces, uppercase for white pieces. numbers represent consequtive spaces. / represents a new row 
    # active color - whose turn it is, either 'w' or 'b'
    # castling rights - which castling moves are still legal K or k for kingside and Q or q for queenside, '-' if no legal castling moves for either player
    # en passant - if the last move was a pawn moving up two squares, this is the space behind the square for the purposes of en passant
    # halfmove clock - number of moves without a pawn move or piece capture, after 50 of which the game is a draw
    # fullmove number - number of full turns starting at 1, increments after black's move

    # Example FEN of starting position
    # rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
    
    parts = re.split(" ", fen)
    piece_placement = re.split("/", parts[0])
    active_color = parts[1]
    castling_rights = parts[2]
    en_passant = parts[3]
    halfmove_clock = int(parts[4])
    fullmove_clock = int(parts[5])

    board_vector = np.zeros( (8,8), dtype=np.float32) # 
    
    # piece to layer structure taken from reference [1]
    piece_to_value = {
        'R': 0.5,
        'N': 0.3,
        'B': 0.35,
        'Q': 0.9,
        'K': 1,
        'P': 0.1,
        'p': -0.1,
        'k': -1,
        'q': -0.9,
        'b': -0.35,
        'n': -0.3,
        'r': -0.5
    }
    
    castling = {
        'K': (7,7),
        'Q': (7,0),
        'k': (0,7),
        'q': (0,0),
    }

    for r, row in enumerate(piece_placement):
        c = 0
        for piece in row:
            if piece in piece_to_value:
                board_vector[r, c] = piece_to_value[piece]
                c += 1
            else:
                c += int(piece)
    
    return board_vector

In [11]:
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
board = fen_to_board_vector(fen)
board=board.flatten()
print(board)



[-0.5  -0.3  -0.35 -0.9  -1.   -0.35 -0.3  -0.5  -0.1  -0.1  -0.1  -0.1
 -0.1  -0.1  -0.1  -0.1   0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.5   0.3   0.35  0.9
  1.    0.35  0.3   0.5 ]


In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)


    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
# ChessDataset code and eval_to_int code taken from reference [1]
class ChessDataset(Dataset):
    def __init__(self, data_frame):
        self.fens = torch.from_numpy(np.array([*map(fen_to_board_vector, data_frame["FEN"])], dtype=np.float32))
        self.evals = torch.Tensor([[x] for x in data_frame["Evaluation"]])
        self._len = len(self.evals)
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, index):
        return self.fens[index], self.evals[index]


def eval_to_int(evaluation):
    try:
        res = int(evaluation)
    except ValueError:
        res = 10000 if evaluation[1] == '+' else -10000
    return res / 100


In [17]:
def AdamW_main():
    MAX_DATA = 100000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}".format(device))

    print("Preparing Training Data...")
    train_data = pd.read_csv("../data/chessData.csv")
    train_data = train_data[:MAX_DATA]
    train_data["Evaluation"] = train_data["Evaluation"].map(eval_to_int)
    trainset = ChessDataset(train_data)
    
    print("Preparing Test Data...")
    test_data = pd.read_csv("../data/tactic_evals.csv")
    test_data = test_data[:MAX_DATA]
    test_data["Evaluation"] = test_data["Evaluation"].map(eval_to_int)
    testset = ChessDataset(test_data)

    batch_size = 10

    print("Converting to pytorch Dataset...")

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


    net = Net().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(net.parameters())


    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                # denominator for loss should represent the number of positions evaluated 
                # independent of the batch size
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / (2000*len(labels))))
                running_loss = 0.0

    print('Finished Training')

    PATH = './chess.pth'
    torch.save(net.state_dict(), PATH)

    print('Evaluating model')

    count = 0
    total_loss = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            #print("Correct eval: {}, Predicted eval: {}, loss: {}".format(labels, outputs, loss))
            
            # count should represent the number of positions evaluated 
            # independent of the batch size
            count += len(labels)
            total_loss += loss
            if count % 10000 == 0:
                print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))
    #print('Average error of the model on the {} tactics positions is {}'.format(count, loss/count))


In [18]:
AdamW_main()

Using device cuda
Preparing Training Data...
Preparing Test Data...
Converting to pytorch Dataset...
[1,  2000] loss: 8.809
[1,  4000] loss: 8.467
[1,  6000] loss: 9.202
[1,  8000] loss: 8.889
[1, 10000] loss: 8.545
[2,  2000] loss: 7.595
[2,  4000] loss: 8.179
[2,  6000] loss: 7.174
[2,  8000] loss: 7.402
[2, 10000] loss: 6.552
[3,  2000] loss: 6.254
[3,  4000] loss: 6.474
[3,  6000] loss: 6.698
[3,  8000] loss: 6.344
[3, 10000] loss: 6.470
[4,  2000] loss: 5.367
[4,  4000] loss: 6.230
[4,  6000] loss: 6.126
[4,  8000] loss: 5.975
[4, 10000] loss: 5.676
[5,  2000] loss: 5.602
[5,  4000] loss: 5.398
[5,  6000] loss: 5.311
[5,  8000] loss: 5.918
[5, 10000] loss: 5.222
[6,  2000] loss: 4.886
[6,  4000] loss: 5.071
[6,  6000] loss: 5.230
[6,  8000] loss: 5.235
[6, 10000] loss: 5.184
[7,  2000] loss: 4.605
[7,  4000] loss: 4.495
[7,  6000] loss: 4.911
[7,  8000] loss: 5.088
[7, 10000] loss: 5.075
[8,  2000] loss: 4.687
[8,  4000] loss: 4.874
[8,  6000] loss: 4.039
[8,  8000] loss: 4.809
[8

In [16]:
%pwd

'/home/mtrang/Documents/rl/Chess-Challenge/devel/notebooks'